# Capstone 2 Project - NYC TLC Trip Record Analysis

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import scipy.stats as stats

In [2]:
pd.set_option('display.max_columns', None)
raw_df = pd.read_csv('NYC_TLC_Trip_Record.csv', low_memory=False)


In [ ]:
import json
with open('app.ipynb', 'r') as f:
    notebook = json.load(f)
print(f"Number of cells: {len(notebook['cells'])}")
print(f"Total size of outputs: {sum(len(str(cell.get('outputs', ''))) for cell in notebook['cells'])}")

# Data cleaning

## Show raw data info

In [ ]:
display(raw_df.describe())
raw_df.info()

df = raw_df.copy()

print("\nNumber of null values summary:")
display(raw_df.isnull().sum())

print("\nNumber of duplicated rows:")
display(df.duplicated().sum())

print("\nZone lookup table:")
# add zone name for pick up and drop-off columns
df_zone = pd.read_csv('taxi_zone_lookup.csv')
display(df_zone)


# Join zone names for both pickup and dropoff locations
df = df.merge(
    df_zone[['LocationID', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
).rename(columns={'Zone': 'PUZone'}).drop('LocationID', axis=1)

df = df.merge(
    df_zone[['LocationID', 'Zone']],
    left_on='DOLocationID',
    right_on='LocationID',
    how='left'
).rename(columns={'Zone': 'DOZone'}).drop('LocationID', axis=1)


print("\ndf with zone names")
display(df)


# Data cleaning analysis


## Several columns have missing data (63887 rows out of 68211 rows)

Columns: 
- `RatecodeID`
- `trip_type`
- `store_and_fwd_flag`
- `passenger_count`
- `payment_type`
- `congestion_surcharge`



## Process missing data for `RatecodeID` and `trip_type`

### `RatecodeID`
- 99 is an invalid data, so we can replace it with NaN first
- Then, we can impute the missing data NaN `RatecodeID` with mode, i.e. 1 (most frequent value) because `RatecodeID` is categorical and the distribution for RatecodeID 1 is 91%


In [ ]:
# CHECK frequency of RatecodeID values
print("Check frequency of RatecodeID:")
display(raw_df['RatecodeID'].value_counts(dropna=False))


print("Check distribution of RatecodeID:")
display(raw_df['RatecodeID'].value_counts(normalize=True, dropna=False))

# show rows where RatecodeID is 99 (invalid data)
display(raw_df[raw_df['RatecodeID'] == 99])


In [ ]:
# First, let's replace 99 with NaN
df['RatecodeID'] = raw_df['RatecodeID'].replace(99, np.nan)

# Now let's check the distribution again
print("RatecodeID Distribution (after removing 99):")
display(df['RatecodeID'].value_counts(normalize=True, dropna=False))


mode_ratecode = df['RatecodeID'].mode()[0]
df['RatecodeID_imputed'] = df['RatecodeID'].fillna(mode_ratecode)

print("\nMode-imputed RatecodeID distribution - no more 99 or NA:")
display(df['RatecodeID_imputed'].value_counts(normalize=True))

# Compare the imputation methods
print("\nComparison of imputation methods:")
comparison = pd.DataFrame({
    'Original': raw_df['RatecodeID'].value_counts(normalize=True, dropna=False),
    'Mode Imputed': df['RatecodeID_imputed'].value_counts(normalize=True),
})
display(comparison)

## `trip_type`
- We observe that rows with `RatecodeID` is 99 (invalid data) also have `trip_type` as null
- We can impute the missing data `trip_type` with mode, i.e. 1 (most frequent value) because `trip_type` is categorical and the distribution for trip_type 1 is already 91.5%

In [ ]:
# check trip_type for null values
print("Trip Type Distribution:")
display(raw_df['trip_type'].value_counts(normalize=True, dropna=False))

null_trip_type = raw_df[raw_df['trip_type'].isnull() & ~(raw_df['store_and_fwd_flag'].isnull())]

print("Trip Type for null values joined with RatecodeID = 99 => it turns out to be the same set of rows")
display(raw_df[raw_df['RatecodeID'] == 99].index)
display(null_trip_type.index)

# we can impute trip_type with mode (most frequent value), which is 1 in this case
df['trip_type_imputed'] = df['trip_type'].fillna(1)

# after imputation, check the distribution
print("Trip Type Frequency after mode imputation:")
display(df['trip_type_imputed'].value_counts(normalize=True, dropna=False))

# Compare the original and imputed distributions
comparison = pd.DataFrame({
    'Original': raw_df['trip_type'].value_counts(normalize=True, dropna=False),
    'Imputed': df['trip_type_imputed'].value_counts(normalize=True),
})

print("Comparison of original and imputed trip_type distributions:")
display(comparison)

# Optionally, you can visualize the comparison
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

comparison['Original'].plot(kind='bar', ax=ax1, title='Original Distribution')
comparison['Imputed'].plot(kind='bar', ax=ax2, title='Imputed Distribution')

plt.tight_layout()
plt.show()


## Data cleaning for `store_and_fwd_flag`
- We can impute the missing data `store_and_fwd_flag` with mode, i.e. N (most frequent value) because `store_and_fwd_flag` is categorical and the distribution for store_and_fwd_flag N is 93.2%
- This column distribution is mostly filled with N, so mode imputation is a good choice

In [ ]:
print("Store and Fwd Flag Frequency:")
display(raw_df['store_and_fwd_flag'].value_counts(dropna=False))

print("Store and Fwd Flag Distribution:")
display(raw_df['store_and_fwd_flag'].value_counts(normalize=True, dropna=False))

mode_store_and_fwd_flag = df['store_and_fwd_flag'].mode()[0]
df['store_and_fwd_flag_imputed'] = df['store_and_fwd_flag'].fillna(mode_store_and_fwd_flag)

print("\nstore_and_fwd_flag distribution:")
display(df['store_and_fwd_flag_imputed'].value_counts(normalize=True))


### Data cleaning for `payment_type`
- There are 4324 null values in `payment_type`
- Since `payment_type` distribution is only 59%, so using mode imputation is not a good idea
- We better use probabilistic imputation to match original distribution as close as possible


In [ ]:
print("Payment Type Frequency:")
display(raw_df['payment_type'].value_counts(dropna=False))

print("Payment Type Distribution:")
display(raw_df['payment_type'].value_counts(normalize=True, dropna=False))

# add probabilistic imputation
# Calculate probabilities excluding NaN
payment_type_probs = raw_df['payment_type'].value_counts(normalize=True).drop(index=np.nan, errors='ignore')
# Normalize probabilities to sum to 1
payment_type_probs = payment_type_probs / payment_type_probs.sum()
nan_mask = df['payment_type'].isna()
random_choices = np.random.choice(payment_type_probs.index, p=payment_type_probs.values, size=nan_mask.sum())

# Create the new column, starting with the original values
df['payment_type_imputed'] = df['payment_type']

# Fill NaN values with the random choices
df.loc[nan_mask, 'payment_type_imputed'] = random_choices

# Verify the result
print("NaN values in payment_type_imputed:", df['payment_type_imputed'].isna().sum())

# Compare distributions
comparison = pd.DataFrame({
    'Original': raw_df['payment_type'].value_counts(normalize=True, dropna=False),
    'Probabilistic': df['payment_type_imputed'].value_counts(normalize=True),
})
print("\nComparison of original and probabilistic imputed distributions:")
display(comparison)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

comparison['Original'].plot(kind='bar', ax=ax1, title='Original Distribution')
comparison['Probabilistic'].plot(kind='bar', ax=ax2, title='Probabilistic Imputed Distribution')

plt.tight_layout()
plt.show()



# Data cleaning for `congestion_surcharge`
- `congestion_surcharge` has 4324 null values
- we notice **strong** positive correlation between `congestion_surcharge` and `DOLocationID`
- since `congestion_surcharge` is categorical with 3 possible values (0, 0.75, 2.5, 2.75), we can use the median value of `congestion_surcharge` for each `DOLocationID` to impute the missing values




In [ ]:
# Data cleaning for `congestion_surcharge`
print("Congestion Surcharge Frequency:")
display(raw_df['congestion_surcharge'].value_counts(dropna=False))

print("Congestion Surcharge Distribution:")
display(raw_df['congestion_surcharge'].value_counts(normalize=True, dropna=False))

raw_df[['congestion_surcharge', 'PULocationID', 'DOLocationID']].corr()

correlation = raw_df[['congestion_surcharge', 'PULocationID', 'DOLocationID']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')


In [ ]:
# Calculate and show median congestion_surcharge for each DOLocationID
median_by_location = df.groupby('DOLocationID')['congestion_surcharge'].median()
display(median_by_location)

# incase there is DOLocationID that have all rows with null `congestion_surcharge`, we can use overall median as fallback
overall_median = df['congestion_surcharge'].median()

# Define a function to get the median congestion_surcharge for a given DOLocationID
# if location_id is not in the dataset, use 0 as default value
def get_median_surcharge(location_id):
    if location_id in median_by_location and not pd.isnull(median_by_location[location_id]):
        return median_by_location[location_id]
    else:
        return overall_median


# Impute each missing values based on respective DOLocationID
df['congestion_surcharge_imputed'] = df.apply(
    lambda row: get_median_surcharge(row['DOLocationID']) if pd.isnull(row['congestion_surcharge']) else row['congestion_surcharge'],
    axis=1
)

# Compare original and imputed distributions
comparison = pd.DataFrame({
    'Original': raw_df['congestion_surcharge'].value_counts(normalize=True, dropna=False),
    'Imputed': df['congestion_surcharge_imputed'].value_counts(normalize=True),
})

print("Comparison of original and imputed congestion_surcharge distributions:")
display(comparison)

comparison = pd.DataFrame({
    'Original': raw_df['congestion_surcharge'].value_counts(normalize=True, dropna=False),
    'Imputed': df['congestion_surcharge_imputed'].value_counts(normalize=True),
})

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# plot original data
comparison['Original'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Original Congestion Surcharge Distribution')
ax1.set_xlabel('Congestion Surcharge')
ax1.set_ylabel('Proportion')
ax1.tick_params(axis='x', rotation=45)

# Plot for imputed data
comparison['Imputed'].plot(kind='bar', ax=ax2, color='lightgreen')
ax2.set_title('Imputed Congestion Surcharge Distribution')
ax2.set_xlabel('Congestion Surcharge')
ax2.set_ylabel('Proportion')
ax2.tick_params(axis='x', rotation=45)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

# Data cleaning for `passenger_count`
- column type is continuous
- better to use regression to impute the missing values

In [ ]:
corr_columns = df.columns.drop(['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'ehail_fee', ])

print("Passenger Count Distribution:")
display(df['passenger_count'].value_counts(normalize=True, dropna=False))

print("\nPassenger Count Summary Statistics:")
display(df['passenger_count'].describe())
# ================================================


## Check normality of `passenger_count`

- Histogram
- Shapiro-Wilk test
- Kolmogorov-Smirnov test

In [ ]:
# Remove NaN values for the analysis
passenger_count = df['passenger_count'].dropna()

# Histogram
plt.figure(figsize=(10, 6))
plt.hist(passenger_count, bins=30, edgecolor='black')
plt.title('Histogram of Passenger Count')
plt.xlabel('Passenger Count')
plt.ylabel('Frequency')
plt.show()

# Shapiro-Wilk test
statistic, p_value = stats.shapiro(passenger_count)
print(f"Shapiro-Wilk test statistic: {statistic}")
print(f"p-value: {p_value}")

if p_value > 0.05:
    print("The passenger_count data appears to be normally distributed (fail to reject H0)")
else:
    print("The passenger_count data does not appear to be normally distributed (reject H0)")

from scipy.stats import kstest

ks_statistic, p_value = kstest(df['passenger_count'].dropna(), 'norm')
print(f"\nKolmogorov-Smirnov test statistic: {ks_statistic}")
print(f"p-value: {p_value}")

if p_value > 0.05:
    print("The data appears to be normally distributed (fail to reject H0)")
else:
    print("The data does not appear to be normally distributed (reject H0)")

In [ ]:
from sklearn.impute import SimpleImputer

# Create a copy of the dataframe for imputation
df_imputed = df.copy()

# Median Imputation
median_imputer = SimpleImputer(strategy='median')
df['passenger_count_imputed'] = median_imputer.fit_transform(df[['passenger_count']])

# Compare distributions after imputation
plt.figure(figsize=(15, 10))

plt.subplot(3, 2, 1)
sns.histplot(df['passenger_count'].dropna(), kde=True)
plt.title('Original (non-null values)')

plt.subplot(3, 2, 2)
sns.histplot(df['passenger_count_imputed'], kde=True)
plt.title('Median Imputation')

plt.tight_layout()
plt.show()

# Print summary statistics
print("Original (non-null values):")
print(df['passenger_count'].dropna().describe())

print("\nMedian Imputation:")
print(df['passenger_count_imputed'].describe())


## - Drop `ehail_fee` column as it does not have any values
## - Convert `lpep_pickup_datetime` and `lpep_dropoff_datetime` to datetime

In [ ]:
df.drop(columns=['ehail_fee'], inplace=True)
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

# get month and year from `lpep_pickup_datetime`
df['month'] = df['lpep_pickup_datetime'].dt.month
df['year'] = df['lpep_pickup_datetime'].dt.year

display(df.describe())
display(df.isnull().sum())

display(df['month'].value_counts())
display(df['year'].value_counts())


# Clean outliers for trip_distance
1. From the summary statistics, we can see that the mean trip_distance is 8.11 miles, and the max is 100 miles
2. But std is 585.11 miles, which is way too high and almost impossible
3. Max trip_distance is 120,098.84 miles which is clearly impossible, because it's larger than the circumference of Earth (~24,901 miles) and NYC's maximum possible trip distance would be around 30-35 miles


In [ ]:
df = df[df['trip_distance'] <= 100]

print("\nCleaned trip distance statistics:")
print(df['trip_distance'].describe())

sns.histplot(data=df, x='trip_distance')


# Fetch NYC Taxi Zone Lookup

In [ ]:
df_zone = pd.read_csv('taxi_zone_lookup.csv')
df_zone.head()


# Create new clean dataframe

- Copy last processed `df`
- Add `duration` column (in minutes)
- Replace null columns with imputed ones
- add `avg_speed` column

In [ ]:
df_clean = df.copy()

df_clean.drop(columns=['store_and_fwd_flag','RatecodeID','passenger_count', 'payment_type','trip_type', 'congestion_surcharge'], inplace=True)
df_clean.rename(columns={'store_and_fwd_flag_imputed':'store_and_fwd_flag', 'RatecodeID_imputed':'RatecodeID', 'passenger_count_imputed':'passenger_count', 'payment_type_imputed':'payment_type', 'trip_type_imputed':'trip_type', 'congestion_surcharge_imputed':'congestion_surcharge'}, inplace=True)

display(df_clean.dtypes)

# derive duration column
df_clean['duration'] = df_clean['lpep_dropoff_datetime'] - df_clean['lpep_pickup_datetime']
df_clean['duration'] = df_clean['duration'].dt.total_seconds() / 60

# add time-related columns
df_clean['hour'] = df_clean['lpep_pickup_datetime'].dt.hour
df_clean['day_of_week'] = df_clean['lpep_pickup_datetime'].dt.day_name()

# No more null values
print("\nNull values summary:")
display(df_clean.isnull().sum())

df_clean

In [ ]:
# Calculate average speed (mph) = distance / (duration in hours)
# Convert duration from minutes to hours by dividing by 60
df_clean['avg_speed'] = df_clean['trip_distance'] / (df_clean['duration'] / 60)

display(df_clean[df_clean['duration'] != 0]['avg_speed'].describe())

# incase duration is 0, replace infinite values with mean, i.e. 13.5 mph
df_clean['avg_speed'] = df_clean['avg_speed'].replace([np.inf, -np.inf], np.nan)

display(df_clean[df_clean['avg_speed'] > 100])


# Exploratory Data Analysis

In [19]:
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Revenue Analysis

In [ ]:
# 1. Revenue Analysis
fig, axes = plt.subplots(2,2, figsize=(20, 17))

# Fare Amount Distribution
sns.histplot(data=df_clean, x='fare_amount', bins=100, ax=axes[0,0])
axes[0,0].set_title('Distribution of Fare Amounts')
axes[0,0].set_xlabel('Fare Amount ($)')
axes[0,0].set_xlim(0, 100)

# Total Amount by Payment Type
sns.boxplot(data=df_clean, x='payment_type', y='total_amount', ax=axes[0,1])
axes[0,1].set_title('Total Amount by Payment Type')
axes[0,1].set_xlabel('Payment Type')
axes[0,1].set_ylabel('Total Amount ($)')

# Average Fare by Location
location_avg_fare = df_clean.groupby('PULocationID')['fare_amount'].mean().sort_values(ascending=False).head(10)
location_avg_fare.plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Top 10 Pickup Locations by Average Fare')
axes[1,0].set_xlabel('Location ID')
axes[1,0].set_ylabel('Average Fare ($)')

# Tips Distribution
sns.histplot(data=df_clean, x='tip_amount', bins=50, ax=axes[1,1])
axes[1,1].set_title('Distribution of Tips')
axes[1,1].set_xlabel('Tip Amount ($)')

plt.tight_layout()
plt.show()

# 2. Trip Efficiency Analysis

In [ ]:
# 2. Trip Efficiency Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Trip Duration vs Fare
sns.scatterplot(data=df_clean, x='duration', y='fare_amount', alpha=0.5, ax=axes[0,0])
axes[0,0].set_title('Trip Duration vs Fare Amount')
axes[0,0].set_xlabel('Duration (minutes)')
axes[0,0].set_ylabel('Fare Amount ($)')

# what plot shall i add to complete 4 subplots?
# Average Speed Distribution
sns.histplot(data=df_clean[(df_clean['duration'] > 0) & (df_clean['avg_speed'] < 100)], x='avg_speed', bins=20, ax=axes[0,1])
axes[0,1].set_xlim(0, 100)
axes[0,1].set_title('Distribution of Average Speeds')
axes[0,1].set_xlabel('Average Speed (mph)')
axes[0,1].set_ylabel('Frequency')


# trip duration vs distance
sns.scatterplot(data=df_clean, x='duration', y='trip_distance', alpha=0.5, ax=axes[1,0])
axes[1,0].set_title('Trip Duration vs Distance')
axes[1,0].set_xlabel('Duration (minutes)')
axes[1,0].set_ylabel('Distance (miles)')

# Passenger Count Distribution
sns.countplot(data=df_clean, x='passenger_count', ax=axes[1,1])
axes[1,1].set_title('Distribution of Passenger Count')
axes[1,1].set_xlabel('Number of Passengers')

plt.tight_layout()
plt.show()


sns.histplot(df_clean['duration'], bins=30, kde=True)
plt.title('Trip Duration Distribution')
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')
plt.show()


# 3. Trip Distance Analysis

In [ ]:
display(df_clean['trip_distance'].describe())
df_clean[df_clean['trip_distance'] > 100]

In [ ]:
# Distance Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Trip Distance Distribution
sns.histplot(data=df_clean, x='trip_distance', bins=50, ax=axes[0,0])
axes[0,0].set_title('Distribution of Trip Distances')
axes[0,0].set_xlabel('Distance (miles)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_xlim(0, 20)  # Focus on typical trip distances

# 2. Distance vs Fare Amount
sns.scatterplot(data=df_clean, x='trip_distance', y='fare_amount', alpha=0.5, ax=axes[0,1])
axes[0,1].set_title('Trip Distance vs Fare Amount')
axes[0,1].set_xlabel('Distance (miles)')
axes[0,1].set_ylabel('Fare Amount ($)')

# 3. Average Distance by Hour
hourly_distance = df_clean.groupby('hour')['trip_distance'].mean()
hourly_distance.plot(kind='line', marker='o', ax=axes[1,0])
axes[1,0].set_title('Average Trip Distance by Hour')
axes[1,0].set_xlabel('Hour of Day')
axes[1,0].set_ylabel('Average Distance (miles)')

# 4. Average Distance by Day
daily_distance = df_clean.groupby('day_of_week')['trip_distance'].mean()
daily_distance.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Average Trip Distance by Day')
axes[1,1].set_xlabel('Day of Week')
axes[1,1].set_ylabel('Average Distance (miles)')

plt.tight_layout()
plt.show()

print("\nDistance Analysis Metrics:")
print(f"Average trip distance: {df_clean['trip_distance'].mean():.2f} miles")
print(f"Median trip distance: {df_clean['trip_distance'].median():.2f} miles")
print(f"Most common trip distance range: {df_clean['trip_distance'].mode().iloc[0]:.2f} miles")
print(f"Maximum trip distance: {df_clean['trip_distance'].max():.2f} miles")


# 4. Time Analysis

In [ ]:
# 3. Time Analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 7))

# Hourly Trip Distribution
hourly_trips = df_clean.groupby('hour')['fare_amount'].mean()
hourly_trips.plot(kind='line', marker='o', ax=axes[0,0 ])
axes[0,0].set_title('Average Fare by Hour of Day')
axes[0,0].set_xlabel('Hour')
axes[0,0].set_ylabel('Average Fare ($)')

# Daily Trip Distribution
daily_trips = df_clean.groupby('day_of_week')['fare_amount'].mean()
daily_trips.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Average Fare by Day of Week')
axes[0,1].set_xlabel('Day of Week')
axes[0,1].set_ylabel('Average Fare ($)')

# Sum of fare_amount by day and hour
hourly_trips = df_clean.groupby('hour')['fare_amount'].sum()
hourly_trips.plot(kind='line', marker='o', ax=axes[1,0])
axes[1,0].set_title('Average Fare by Hour of Day')
axes[1,0].set_xlabel('Hour')
axes[1,0].set_ylabel('Average Fare ($)')

daily_trips = df_clean.groupby('day_of_week')['fare_amount'].sum()
daily_trips.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Average Fare by Day of Week')
axes[1,1].set_xlabel('Day of Week')
axes[1,1].set_ylabel('Average Fare ($)')

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(2,1,figsize=(10, 8))

# Hourly Passenger Count
hourly_passengers = df_clean.groupby('hour')['passenger_count'].mean()
hourly_passengers.plot(kind='line',
                      marker='o',
                      ax=axes[0])
axes[0].set_title('Average Passengers by Hour')

# Revenue by Day and Hour, using total_amount
pivot_data = df_clean.pivot_table(index='hour',
                                 columns='day_of_week',
                                 values='total_amount',
                                 aggfunc='mean')
sns.heatmap(pivot_data, 
            cmap='YlOrRd',
            ax=axes[1])
axes[1].set_title('Average Revenue by Day and Hour')

# add legend for heatmap
cbar = axes[1].collections[0].colorbar
cbar.set_label('Average Revenue ($)')

plt.tight_layout()
plt.show()

# 5. Driver satisfaction

In [ ]:
# subplots
fig, axes = plt.subplots(3,2, figsize=(14, 10))
sns.scatterplot(data=df_clean, 
                x='fare_amount',
                y='tip_amount',
                alpha=0.5,
                ax=axes[0,0])
axes[0,0].set_title('Fare Amount vs Tip Amount')

# analyze tips by hour
df_clean.groupby('hour')['tip_amount'].mean().plot(kind='line', marker='o', ax=axes[0,1])
axes[0,1].set_title('Average Tips by Hour')
axes[0,1].set_xlabel('Hour')
axes[0,1].set_ylabel('Average Tips ($)')


# analyze tips by day of week
df_clean.groupby('day_of_week')['tip_amount'].mean().plot(kind='bar', ax=axes[  1,0])
axes[1,0].set_title('Average Tips by Day of Week')
axes[1,0].set_xlabel('Day of Week')
axes[1,0].set_ylabel('Average Tips ($)')


# analyze tips by passenger count
df_clean.groupby('passenger_count')['tip_amount'].mean().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Average Tips by Passenger Count')
axes[1,1].set_xlabel('Passenger Count')
axes[1,1].set_ylabel('Average Tips ($)')

# analyze tips by PULocationID
df_clean.groupby('PULocationID')['tip_amount'].mean().sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[2,0])
axes[2,0].set_title('Top 10 Pickup Locations by Average Tips')
axes[2,0].set_xlabel('Location ID')
axes[2,0].set_ylabel('Average Tips ($)')

# analyze tips by DOLocationID
df_clean.groupby('DOLocationID')['tip_amount'].mean().sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[2,1])
axes[2,1].set_title('Top 10 Dropoff Locations by Average Tips')
axes[2,1].set_xlabel('Location ID')
axes[2,1].set_ylabel('Average Tips ($)')


plt.tight_layout()
plt.show()


# Location analysis

In [26]:
import geopandas as gpd
import folium
from folium import plugins
import branca.colormap as cm

# Read the shapefile
nyc_zones = gpd.read_file('taxi_zones_spatial/taxi_zones.shp')

In [ ]:
zone_stats = df_clean.groupby('PULocationID')['fare_amount'].mean().reset_index()
nyc_zones = nyc_zones.merge(zone_stats, left_on='OBJECTID', right_on='PULocationID', how='left')

# Start with New York location
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# Create a colormap
colormap = cm.LinearColormap(
    colors=['yellow', 'red'],
    vmin=zone_stats['fare_amount'].min(),
    vmax=zone_stats['fare_amount'].max()
)
m.add_child(colormap)

# Add GeoJson data to folium map
folium.GeoJson(
    nyc_zones,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['fare_amount']) if feature['properties']['fare_amount'] else 'grey',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['zone', 'fare_amount', 'borough'],
        aliases=['Zone:', 'Avg Fare:', 'Borough:'],
        localize=True
    )
).add_to(m)

# Save the map
display(m)

## Top 10 routes with highest number of trips


In [ ]:
import geopandas as gpd
from folium import plugins


# Convert the coordinate system to lat/long (EPSG:4326)
nyc_zones_2 = nyc_zones.to_crs(epsg=4326)

# Calculate top 10 routes based on frequency
top_routes = df_clean.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='count')
top_routes = top_routes.sort_values('count', ascending=False).head(10)

# Merge with zone names
top_routes = top_routes.merge(
    df_clean[['PULocationID', 'PUZone']].drop_duplicates(),
    on='PULocationID'
).merge(
    df_clean[['DOLocationID', 'DOZone']].drop_duplicates(),
    on='DOLocationID'
)

# Now create the map
m = folium.Map(location=[40.785091, -73.878284], zoom_start=12)
# generate 10 different colors
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'pink', 'brown', 'gray', 'cyan']


unique_pickup_zones = top_routes['PULocationID'].unique()
unique_dropoff_zones = top_routes['DOLocationID'].unique()
all_involved_zones = np.union1d(unique_pickup_zones, unique_dropoff_zones)

# Add zone polygons of each used zones
for zone_id in all_involved_zones:
    try:
        zone_geom = nyc_zones_2[nyc_zones_2['OBJECTID'] == zone_id]
        if not zone_geom.empty:
            # Add zone polygon with light fill and border
            folium.GeoJson(
                zone_geom,
                style_function=lambda x: {
                    'fillColor': '#0000FF', 
                    'color': '#000000', 
                    'weight': 2,
                    'fillOpacity': 0.3
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=['zone'],
                    aliases=['Zone:'],
                    localize=True
                )
            ).add_to(m)
    except Exception as e:
        print(f"Error processing zone {zone_id}: {e}")

# add the routes
for idx, route in top_routes.iterrows():
    try:
        # Get coordinates
        pu_geom = nyc_zones_2[nyc_zones_2['OBJECTID'] == route['PULocationID']]['geometry'].iloc[0]
        do_geom = nyc_zones_2[nyc_zones_2['OBJECTID'] == route['DOLocationID']]['geometry'].iloc[0]
        
        # get x,y coordinates
        pu_lat, pu_lon = pu_geom.centroid.y, pu_geom.centroid.x
        do_lat, do_lon = do_geom.centroid.y, do_geom.centroid.x
        current_color = colors[idx % len(colors)]

        # Add pickup marker
        folium.CircleMarker(
            location=[pu_lat, pu_lon],
            radius=5,
            color=current_color,
            fill=True,
            fillOpacity=1.0,
            popup=f"Pickup: {route['PUZone']}\nTrips: {route['count']}",
            weight=3
        ).add_to(m)
        
        # add dropoff marker
        folium.CircleMarker(
            location=[do_lat, do_lon],
            radius=5,
            color=current_color,
            fill=True,
            fillOpacity=1.0,
            popup=f"Dropoff: {route['DOZone']}\nTrips: {route['count']}",
            weight=3
        ).add_to(m)
        
        # Add line
        line = folium.PolyLine(
            locations=[[pu_lat, pu_lon], [do_lat, do_lon]],
            color=current_color,
            weight=4,
            opacity=1.0,
            popup=f"Route {idx+1}: {route['count']} trips"
        ).add_to(m)

        # add arrow icon on the line
        plugins.PolyLineTextPath(
            line,
            text='➜',
            repeat=True,
            offset=8,
            attributes={'font-size': '24px', 'fill': current_color}
        ).add_to(m)


    except Exception as e:
        print(f"Error processing route {idx+1}: {e}")

# add shape for the zones used

# legend
legend_html = f'''
<div style="position: fixed; 
            top: 20px; right: 20px; width: 200px;
            border: 1px solid rgba(0,0,0,0.2);
            border-radius: 4px;
            z-index: 9999; 
            background-color: rgba(255,255,255,0.9);
            padding: 8px;  
            font-size: 11px;">
     <p style="margin: 0 0 5px 0;"><b>Top {len(top_routes)} Routes</b></p>
     <div style="max-height: 200px; overflow-y: auto;">
'''

#
for idx, route in top_routes.iterrows():
    current_color = colors[idx % len(colors)]

    legend_html += f'''
    <div style="margin-bottom: 5px;">
        <i class="fa fa-circle fa-1x" style="color:{current_color}"></i> Route {idx+1}: {route['count']}<br>
        <span style="margin-left: 12px">From: {route['PUZone']}</span><br>
        <span style="margin-left: 12px">To: {route['DOZone']}</span>
    </div>
    '''

legend_html += '''
     </div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
display(m)

## Insights
1. The highest number of trips are from East Harlem North to East Harlem South. It's 


## Save the cleaned dataframe

In [29]:
df_clean.to_csv('NYC_TLC_cleaned_zone.csv', index=False)